# Import

import packages

In [1]:

import numpy as np
import pandas as pd

## visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from plotnine import *
import calmap

import plotly.express as px
import folium

## color pallette
cdr = ['#393e46', '#ff2e63', '#30e3ca'] # grey - red - blue
idr = ['#f8b400', '#ff2e63', '#30e3ca'] # yellow - red - blue

import datasets

In [2]:

# !ls -lt Coronavirus/dataset/covid_19_clean_complete.csv
full_table = pd.read_csv('dataset/covid_19_clean_complete.csv', parse_dates=['Date'])

## checking for missing value
# full_table.isna().sum()

## dataframe info
# dir(full_table)
full_table.head(3)
# full_table.info()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.8257,117.2264,2020-01-22,1,0,0
1,Beijing,Mainland China,40.1824,116.4142,2020-01-22,14,0,0
2,Chongqing,Mainland China,30.0572,107.8740,2020-01-22,6,0,0


# Preprocessing

cleaning data


In [3]:

case_list = ['Confirmed', 'Deaths', 'Recovered', 'Active']

## Active Case = Confirmed - Deaths - Recovered
full_table[case_list[3]] = full_table[case_list[0]] - full_table[case_list[1]] - full_table[case_list[2]]

## filling missing values with NA
na_str = 'NA'
full_table[case_list] = full_table[case_list].fillna(na_str)
full_table.iloc[:,[0]] = full_table[['Province/State']].fillna(na_str)

## replacing Mainland china with just China
full_table.iloc[:,[1]] = full_table[['Country/Region']].replace('Mainland China', 'China')

full_table[full_table.iloc[:,1] == 'Algeria'].sort_values(by = case_list, ascending = False).head(3)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
7354,NA,Algeria,28.0339,1.6596,2020-03-04,12,0,0,12
7527,NA,Algeria,28.0339,1.6596,2020-03-05,12,0,0,12
7181,NA,Algeria,28.0339,1.6596,2020-03-03,5,0,0,5


derived tables

In [4]:

# cases in the ships: the 'Grand Princess' condition isn't helpful
ship = full_table[full_table['Province/State'].str.contains('Grand Princess')|full_table['Country/Region'].str.contains('Cruise Ship')]
# temp_all = full_table[full_table['Province/State'].str.contains('Grand Princess')]
# temp_g = full_table[full_table['Province/State'].str.contains('Grand')]
# temp_p = full_table[full_table['Province/State'].str.contains('Princess')]

# print('instences from Grand or Princess: {all_num}\ninstences from Grand: {g_num}\ninstences from Princess: {p_num}'.format(all_num = len(temp_all.index), g_num = len(temp_g.index), p_num = len(temp_p.index)))

# china and the row
china_filter = full_table['Country/Region'] == 'China'
china = full_table[china_filter]
row = full_table[china_filter == False]

# latest
full_latest = full_table[full_table['Date'] == max(full_table['Date'])].reset_index()
china_latest = china[china['Date'] == max(china['Date'])].reset_index()
row_latest = row[row['Date'] == max(row['Date'])].reset_index()

#latest condensed
full_latest_grouped = full_latest.groupby('Country/Region')[case_list].sum().reset_index()
china_latest_grouped = china_latest.groupby('Country/Region')[case_list].sum().reset_index()
row_latest_grouped = row_latest.groupby('Country/Region')[case_list].sum().reset_index()

# Lateset Data

latest complete data

In [5]:

## show the latest record with the max cumulative numbers of each colunm and group 
temp = full_table.groupby(['Country/Region', 'Province/State'])[case_list].max()

# temp.style.background_gradient(cmap = 'Reds')

latest condensed data

In [6]:
temp = full_table.groupby('Date')[case_list].sum().reset_index()
temp = temp[temp['Date'] == max(temp['Date'])].reset_index(drop = True)
temp['Global Mortality'] = temp['Deaths']/temp['Confirmed']
temp['Deaths per 100 Confirmed Cases'] = temp['Global Mortality'] * 100
temp.style.background_gradient(cmap = 'Pastel1')

,Date,Confirmed,Deaths,Recovered,Active,Global Mortality,Deaths per 100 Confirmed Cases
0,2020-03-05 00:00:00,97886,3348,53797,40741,0.034203,3.420305


In [7]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected = True)

In [8]:
tm = temp.melt(id_vars = 'Date', value_vars = ['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path = ['variable'], values = 'value', height = 400, width = 600)
fig.data[0].textinfo = 'label+text+value'
fig.show()